In [1]:
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold


import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 80)

In [2]:
df= pd.read_csv('train_processed.csv', low_memory=False)
df_test= pd.read_csv('test_processed.csv', low_memory=False)



In [3]:
if 'Unnamed: 0' in df.columns:
    del df['Unnamed: 0']
    
if 'Unnamed: 0' in df_test.columns:
    del df_test['Unnamed: 0']

In [14]:
list(df.columns.values)

In [15]:
X_features = list(df.columns.values)
X_features.remove('charge_off')
X = df[X_features]
y = df['charge_off']

X_test = df_test[X_features]
y_test = df_test['charge_off']

In [21]:
clf_results = {}
kf = StratifiedKFold(n_splits=5,random_state=1, shuffle= True )
empty_df = pd.DataFrame(columns = ['1','2', '3'], index = ['0.1','0.3', '0.5', '0.7'])

recall , precision , accuracy = empty_df.copy(), empty_df.copy(), empty_df.copy()
f1score, roc_auc, logistic_loss = empty_df.copy(), empty_df.copy(), empty_df.copy()

split = 1
for train_index,test_index in kf.split(X,y):
    
    print ('\n{} of kfold {} '.format(split,kf.n_splits))
    
    x_train,x_val = X.loc[train_index],X.loc[test_index]
    y_train,y_val = y[train_index],y[test_index]
    
    model = LogisticRegression(random_state=1)
    model.fit(x_train,y_train)
    
    pred_test=model.predict_proba(x_val)
    for thr in [0.1, 0.3, 0.5, 0.7]:
        pred_new = np.where(pred_test[:,1] > thr , 1 , 0 )
        recall.loc[str(thr), str(split)] = recall_score(y_val,pred_new)
        precision.loc[str(thr), str(split)] = precision_score(y_val,pred_new)
        accuracy.loc[str(thr), str(split)] = accuracy_score(y_val,pred_new)
        f1score.loc[str(thr), str(split)] = f1_score(y_val,pred_new)
        roc_auc.loc[str(thr), str(split)] = roc_auc_score(y_val,pred_new)
        logistic_loss.loc[str(thr), str(split)] = log_loss(y_val,pred_new)
        
    
    split = split + 1 
    

clf_results['Logistic Regression_2'] = [recall , precision , accuracy , f1score, roc_auc, logistic_loss]
   


1 of kfold 5 

2 of kfold 5 

3 of kfold 5 

4 of kfold 5 

5 of kfold 5 


In [22]:
recall

,1,2,3,4,5
0.1,0.230711,0.237883,0.268963,0.284721,0.257472
0.3,0.00434688,0.00510759,0.00641165,0.004999,0.005652
0.5,0,0,0,0.000000,0.000000
0.7,0,0,0,0.000000,0.000000


In [23]:
precision

,1,2,3,4,5
0.1,0.0600192,0.0595808,0.0616101,0.064674,0.060573
0.3,0.101781,0.12336,0.140143,0.117949,0.139785
0.5,0,0,0,0.000000,0.000000
0.7,0,0,0,0.000000,0.000000


In [24]:
accuracy

,1,2,3,4,5
0.1,0.772769,0.765805,0.74969,0.749414,0.754484
0.3,0.946387,0.946534,0.946443,0.946472,0.946646
0.5,0.948145,0.948151,0.948151,0.948151,0.948150
0.7,0.948151,0.948151,0.948151,0.948151,0.948156


In [25]:
f1score

,1,2,3,4,5
0.1,0.0952573,0.0952941,0.100255,0.105405,0.098073
0.3,0.00833768,0.00980904,0.0122623,0.009591,0.010864
0.5,0,0,0,0.000000,0.000000
0.7,0,0,0,0.000000,0.000000


In [26]:
logistic_loss

,1,2,3,4,5
0.1,7.84842,8.08897,8.64557,8.655104,8.479997
0.3,1.85172,1.84666,1.84978,1.848806,1.842783
0.5,1.791,1.79081,1.79081,1.790810,1.790820
0.7,1.79081,1.79081,1.79081,1.790810,1.790626
